In [ ]:
p_exact = cbrt(25)
a = 2;
b = 3;
TOL = 10^-8;

@show n_max= ceil(Int, log2((b-a)/TOL));

In [ ]:
f(x) = x^3 - 25
p = 0;
for i in 1:n_max
    p = 0.5 *  (a+b);
    if ( f(a) * f(p)<=0)
        b = p;
    else
        a = p
    end
    println("$i: |p - p_exact| = $(round(abs(p - p_exact),digits=12))")
end

#### I estimated that we would need 27 iterations to get a tolerance of 10^-8, but it only took about 22 iterations as shown above.

In [ ]:
function newton(f, df, p0, n_max, rel_tol; verbose = true)
    
    converged = false;
    p = p0;
    p_old = p0;

    for i in 1:n_max

        p = p_old - f(p_old)/df(p_old);
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end

        if (i>1)
            if abs(p-p_old)/abs(p)< rel_tol
                converged = true;
                break
            end
        end

        p_old = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

In [ ]:
import Pkg; Pkg.add("ForwardDiff")

In [ ]:

using ForwardDiff

f2 = x-> x^3 - 25;
df_auto = x-> ForwardDiff.derivative(f2,x)
p0 = 3;
rel_tol = 10^-8;
n_max = 100;

p = newton(f2, df_auto, p0, n_max, rel_tol);

#### It takes 3 iterations to get to the toleranc eof 10^-8

In [ ]:
# False Position Method in Julia
function false_position(f, a, b, tol=1e-8, max_iter=100)
    fa = f(a)
    fb = f(b)
    if fa * fb > 0
        error("Function must have opposite signs at endpoints a and b.")
    end

    iter = 0
    c_old = a
    while iter < max_iter
        # Compute the false position
        c = b - fb*(b - a)/(fb - fa)
        fc = f(c)
        # Compute the absolute error
        err = abs(c - c_old)
        println("Iteration $iter: c = $c, error = $err")
        if err < tol
            return c
        end
        # Update interval
        if fa * fc < 0
            b = c
            fb = fc
        else
            a = c
            fa = fc
        end
        c_old = c
        iter += 1
    end
    println("Maximum iterations reached.")
    return c
end

# Define the function f(x) = x^3 - 25
f(x) = x^3 - 25

# Apply false position method on interval [2,3] with tolerance 1e-8
root = false_position(f, 2, 3, 1e-8)
println("Estimated root: $root")
println("Julia cubic root for comparison: ", cbrt(25))


Some q's:
How does the significant digits change throughout each iteration for these methods?

Bisection: The number of significant digits changes relatively slow ocmpared to Newton's and the Method of False Position.

Newton's: The number of significant digits changes very quickly in this method.

Method of False Position: The number of significant digits changes quickly, but not as quick as Newton's method.



In [ ]:
using Plots

# Define the polynomial
f(x) = 230x^4 + 18x^3 + 9x^2 - 221x - 9

# Create x values in the interval [-1, 1]
x = LinRange(-1, 1, 500)

# Plot the function with grid
plot(x, f.(x), label="f(x)", lw=2, grid=true)
hline!([0], color=:black, linestyle=:dash, label="y=0") # x-axis
xlabel!("x")
ylabel!("f(x)")
title!("Plot of f(x) = 230x^4 + 18x^3 + 9x^2 - 221x - 9")


In [ ]:
using ForwardDiff

f3 = x-> 230x^4 + 18x^3 + 9x^2 - 221x - 9
df_auto = x-> ForwardDiff.derivative(f3,x)
p0 = -.5;
rel_tol = 10^-6;
n_max = 100;

println("Using p0 = -.5")
p = newton(f3, df_auto, p0, n_max, rel_tol);
println("Using p0 = 1")
p2 = newton(f3, df_auto, 1, n_max, rel_tol);

In [ ]:
function secant(f, p0, p1, n_max, rel_tol; verbose = true)
    
    converged = false;
    
    p = p0;
    for i in 1:n_max

        p = p1 - f(p1) * (p1-p0)/(f(p1)-f(p0));
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end

        
        if (i>1)
            if abs(p-p1)/abs(p1)< rel_tol
                converged = true;
                break
            end
        end
        p0 = p1;
        p1 = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

In [ ]:
fs = x-> 230x^4 + 18x^3 + 9x^2 - 221x - 9
p0 = 1;
p1 = 2;
rel_tol = 10^-6;
n_max = 100;

println("Secant with p0 = 1, p1 = 2")
p = secant(fs, p0, p1, n_max, rel_tol);
println("Secant with p0 = -1, p1 = 0")
p = secant(fs, -1, 0, n_max, rel_tol);

In [ ]:
function muller(f, p0, p1, p2, n_max, rel_tol; verbose = true)
    
    converged = false;
    p = p2;

    for i in 1:n_max

        # solve for the constants a, b, and c
        c = f(p2);
        A = [(p0-p2)^2 p0-p2; (p1-p2)^2 p1-p2 ]; # builds matrix
        x = A\[f(p0)-c; f(p1)-c]; # solves for a and b
        a = x[1];
        b = x[2];
        
        # take the root with larger denominator
        if abs(b + sqrt(b^2-4*a*c))> abs(b - sqrt(b^2-4*a*c))
            p = p2 - 2*c/(b + sqrt(b^2-4*a*c));
        else
            p = p2 - 2*c/(b - sqrt(b^2-4*a*c));            
        end
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end
        
        if (i>1)
            if abs(p-p2)/abs(p)< rel_tol
                converged = true;
                break
            end
        end

        # update entries
        p0 = p1;
        p1 = p2;
        p2 = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

In [ ]:
fm = x -> 230x^4 + 18x^3 + 9x^2 - 221x - 9

p0 = .1
p1 = 0
p2 = -.1

rel_tol = 1e-8;
n_max = 100;

println("Root 1")
p = muller(fm, p0, p1, p2, n_max, rel_tol);


p0 = .9;
p1 = 1;
p2 = 1.5;
println("Root 2")
p = muller(fm, p0, p1, p2, n_max, rel_tol);

p0 = -0.5 + 0.7im
p1 = -0.4 + 0.9im
p2 = -0.6 + 0.7im
println("Root 3")
root3 = muller(fm, p0, p1, p2, n_max, rel_tol)

p0 = -0.5 - 0.7im
p1 = -0.4 - 0.9im
p2 = -0.6 - 0.7im
println("Root 4")
root4 = muller(fm, p0, p1, p2, n_max, rel_tol)


In [ ]:
function fixed_point_iteration(g, p, n_max, rel_tol; verbose=true)
    p_old = p;
    for i in 1:n_max
        p = g(p);
        println("$i: p = $p");
        if(i>1)
            if abs(p_old-p)/abs(p) < rel_tol
                break;
            end
        end
        p_old = p;
    end

    return p
    
end

In [ ]:

fa = p -> p * (1 + (7 - p^5) / (p^2))
fb = p -> p - ((p^5 - 7) / (p^2))
fc = p -> p - ((p^5 - 7) / (5*(p^4)))
fd = p -> p-((p^5-7)/12)
a = fixed_point_iteration(fa, 1, 10, 10^-3)
b = fixed_point_iteration(fb, 1, 10, 10^-3)
c = fixed_point_iteration(fc, 1, 10, 10^-3)
d = fixed_point_iteration(fd, 1, 10, 10^-3)


Ranking from speed of convergence with p_0 = 1, it would be C, D, B, A 